In [1]:
import pandas as pd
from tqdm import tqdm
import time
from splinter import Browser
from bs4 import BeautifulSoup as bs

In [2]:
#*   Prohibits truncatation of data in prints.
pd.set_option('display.max_colwidth', -1)

In [3]:
titleBasics = pd.read_csv("/Users/nicolespaar/Desktop/oscarDataAnalysis/Spaar/resources/data/title.basics.tsv", sep='\t', header=0, low_memory=False)

#todo:::   this should be a function
lowercased_titles = []
count = 0
for row in titleBasics.iterrows():
    try:
        lower = row[1].primaryTitle.lower()
        lowercased_titles.append(lower)
    except:
        count += 1
        lowercased_titles.append("n/a")
print(f"{count} movies had no 'primaryTitle'")
titleBasics['lowercasedTitle'] = lowercased_titles

titleBasics.head(3)

7 movies had no 'primaryTitle'


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,lowercasedTitle
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short",carmencita
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short",le clown et ses chiens
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance",pauvre pierrot


In [4]:
titleCrew = pd.read_csv("/Users/nicolespaar/Desktop/oscarDataAnalysis/Spaar/resources/data/title.crew.tsv", sep='\t', header=0)
titleCrew.head(3)

,tconst,directors,writers
0,tt0000001,nm0005690,\N
1,tt0000002,nm0721526,\N
2,tt0000003,nm0721526,\N


In [5]:
nameBasics = pd.read_csv("/Users/nicolespaar/Documents/name.basics.tsv", sep='\t', header=0, index_col='nconst')
nameBasics.head(3)

,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
nconst,,,,,
nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0072308,tt0053137,tt0050419,tt0043044"
nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0071877,tt0037382,tt0117057,tt0038355"
nm0000003,Brigitte Bardot,1934,\N,"actress,soundtrack,producer","tt0059956,tt0057345,tt0054452,tt0049189"


In [6]:
titlePrincipals = pd.read_csv("/Users/nicolespaar/Desktop/oscarDataAnalysis/Spaar/resources/data/title.principals.tsv", sep='\t', header=0, index_col='nconst')
titlePrincipals.head(3)

,tconst,ordering,category,job,characters
nconst,,,,,
nm1588970,tt0000001,1,self,\N,"[""Herself""]"
nm0005690,tt0000001,2,director,\N,\N
nm0374658,tt0000001,3,cinematographer,director of photography,\N


In [23]:
#*   User input of which movie to munge attributes for then predict against the model. The
#*   input is turned to all lower case to account for any sort of user casing input.
movieToSearch = str(input("Enter a movie title: ")).lower()  #"Toy Story 4"

Enter a movie title: Rain Man


In [24]:
def bugFixer(affectedList):
    '''
    bugFixer() receives a python list/array and checks each item to see if its 0th character
    is whitespace. If it is, the whitespace is removed and appended to a new list. If it is
    not, the item is appended to the same new list. The new list with no items' first
    character being whitespace is then returned.
    '''
    fixedList = []
    for item in affectedList:
        if item[0] == " ":
            fixedItem = item[1:]
            fixedList.append(fixedItem)
        else:
            fixedList.append(item)
    return fixedList

def lengths(x):
    if isinstance(x, list):
        yield len(x)
        for y in x:
            yield from lengths(y)

def iAmount(anArray):
    return max(lengths(anArray))

def init_splinter():
    '''
    init_splinter() initializes a selenium chrome webdriver using a Tor proxy and returns it
    as a splinter browser wrapped object.
    '''
    executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
    browser = Browser('chrome', **executable_path, headless=False, incognito=True)
    return browser

def simmer_soup():
    '''
    simmer_soup() receives the browser object and returns the current page's parsed html as "Soup".
    '''
    html = browser.html
    soup = bs(html, 'html.parser')
    return html, soup

#!---- ---- ---- ---- If $movieToSearch exists in db ---- ---- ---- ----!#
def tconstMunge():
    tconst = titleBasicsRow.tconst.to_string(index=False)[1:]
    #todo:::   split off at first comma if exists (some have multiple tconsts?)
    return tconst

def genresMunge():
    genres_raw = titleBasicsRow.genres.to_string(index=False).split(",")
    genres = bugFixer(genres_raw)
    return genres

def entitiesMunge():
    titleCrewRow = titleCrew[titleCrew['tconst'] == tconst]
    crew_directors = titleCrewRow.directors.to_string(index=False).split(",")
    crew_writers = titleCrewRow.writers.to_string(index=False).split(",")
    titlePrincipalsRow = titlePrincipals[titlePrincipals['tconst'] == tconst]
    
    cast_actors = []
    cast_actors_jobs = []
    cast_other = []
    cast_other_jobs = []
    
    for row in titlePrincipalsRow.iterrows():
        x = row[1].category
        if (x in ('actor', 'actress')):
            cast_actors.append(row[0])
            
            cast_actors_jobs.append(row[1].job)
        else:
            cast_other.append(row[0])
            cast_other_jobs.append(row[1].job)
            
    return cast_actors, cast_actors_jobs, cast_other, cast_other_jobs, crew_directors, crew_writers
    
def entitiesMunge_Parser(cast_actors, cast_other, crew_directors, crew_writers):
    actor_names = []
    cast_names = []
    director_names = []
    writer_names = []
    
    for i in cast_actors:
        actor_names.append(nameBasics.loc[i].primaryName)
    for i in cast_other:
        cast_names.append(nameBasics.loc[i].primaryName)
    for i in crew_directors:
        director_names.append(nameBasics.loc[i].primaryName)
    for i in crew_writers:
        writer_names.append(nameBasics.loc[i].primaryName)
    
    return actor_names, cast_names, director_names, writer_names    
    
#!---- ---- ---- ---- If $movieToSearch does NOT exist in db ---- ---- ---- ----!#



In [25]:
#!---- ---- ---- ---- $movieToSearch regardless of existensce in db ---- ---- ---- ----!#


In [26]:
#   Try to match the user's input movie title to an existing row in the IMDB dataset. If it is
#   found, the row is isolated and its tconst and genres are extracted.
titleBasicsArray = titleBasics[titleBasics['lowercasedTitle'] == movieToSearch]
titleBasicsRow = titleBasicsArray[titleBasicsArray['titleType'] == 'movie']
try:
    #todo:::   this `try` should all be a function
    tconst = tconstMunge()
    
    ################# This won't work in its own function #################
    tconst = tconstMunge()
    imdbURL_base = "https://www.imdb.com/title/"
    imdbURL = str(imdbURL_base + tconst)

    browser = init_splinter()
    browser.visit(imdbURL)
    html, soup = simmer_soup()
    
    attribute_parents = soup.find_all('div', class_='txt-block')
    #todo:::   this isn't always the same, need to account for changes
    prod_companies_parent = attribute_parents[-9]
    links = prod_companies_parent.find_all('a')
    
    prod_companies_raw = []
    for i in links:
        prod_companies_raw.append(i.text[1:])
        
    browser.quit()
    
    prod_companies = prod_companies_raw[0:-1]  
    ################# ################# #################
    
    genres = genresMunge()
    cast_actors, cast_actors_jobs, cast_other, cast_other_jobs, crew_directors, crew_writers = entitiesMunge()
    cast_actors, cast_other, crew_directors, crew_writers = bugFixer(cast_actors), bugFixer(cast_other), bugFixer(crew_directors), bugFixer(crew_writers)      
    actor_names, cast_names, director_names, writer_names = entitiesMunge_Parser(cast_actors, cast_other, crew_directors, crew_writers)    
    print("Movie Found!\n")
    print("TITLE::: ", movieToSearch, "\n\nTCONST:::   ", tconst, "\n\nGENRES:::   ",
             genres, "\n\nACTORS' NCONSTS:::   ", cast_actors, "\n\nACTORS' NAMES:::   ", actor_names, 
          "\n\nACTORS' JOBS:::   ", cast_actors_jobs, "\n\nNON-ACTORS' NCONSTS:::   ", cast_other,
          "\n\nNON-ACTORS' NAMES:::   ", cast_names, "\n\nNON-ACTORS' JOBS:::   ", cast_other_jobs, 
          "\n\nDIRECTORS' NCONSTS:::   ", crew_directors, "\n\nDIRECTOS' NAMES:::   ", director_names,
          "\n\nWRITERS' NCONSTS:::   ", crew_writers, "\n\nWRITERS' NAMES:::   ", writer_names,
          "\n\nPRODUCTION COMPANIES:::   ", prod_companies)
except:
    #todo:::   this `except` should all be a function
    print("[!] Movie not Found, Searching Interwebs..")





Movie Found!

TITLE:::  rain man 

TCONST:::    tt0095953 

GENRES:::    ['Drama'] 

ACTORS' NCONSTS:::    ['nm0000163', 'nm0000129', 'nm0000420', 'nm0596520'] 

ACTORS' NAMES:::    ['Dustin Hoffman', 'Tom Cruise', 'Valeria Golino', 'Gerald R. Molen'] 

ACTORS' JOBS:::    ['\\N', '\\N', '\\N', '\\N'] 

NON-ACTORS' NCONSTS:::    ['nm0005868', 'nm0001469', 'nm0607454', 'nm0060103', 'nm0425741', 'nm0001877'] 

NON-ACTORS' NAMES:::    ['John Seale', 'Barry Levinson', 'Barry Morrow', 'Ronald Bass', 'Mark Johnson', 'Hans Zimmer'] 

NON-ACTORS' JOBS:::    ['director of photography', '\\N', 'story', 'screenplay', 'producer', '\\N'] 

DIRECTORS' NCONSTS:::    ['nm0001469'] 

DIRECTOS' NAMES:::    ['Barry Levinson'] 

WRITERS' NCONSTS:::    ['nm0607454', 'nm0060103'] 

WRITERS' NAMES:::    ['Barry Morrow', 'Ronald Bass'] 

PRODUCTION COMPANIES:::    []


In [18]:
print(imdbURL)

https://www.imdb.com/title/tt0018578
 tt0061196
 tt1478874
 tt2284790
 tt2406100
 tt5872368
 tt9662720
